<a href="https://colab.research.google.com/github/yohanesnuwara/reservoir-engineering/blob/master/Unit%203%20Reservoir%20Statics/notebook/3_problems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Unit 3 Reservoir Statics (Problems)**

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
!git clone https://github.com/yohanesnuwara/reservoir-engineering

Cloning into 'reservoir-engineering'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 203 (delta 16), reused 0 (delta 0), pack-reused 158
Receiving objects: 100% (203/203), 225.03 KiB | 1.23 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [0]:
import sys, os
sys.path.append('/content/reservoir-engineering/Unit 3 Reservoir Statics/functions')

# Problem 3.1

In [10]:
# Question 1. Determine oil density gradient for case 1

# known

so = 0.8 # stock-tank oil specific gravity
sg = 0.65 # solution-gas specific gravity
Rs = 940 # scf/STB
Bo = 1.63

# standard conditions

temp_sc = 60 # in fahrenheit
pressure_sc = 14.73 # 1 atm = 14.73 psia
z_sc = 1 # gas z factor at standard condition

def rhooilgrad(so, sg, Rs, Bo):
  Rs_converted = 940 * (1/5.6148) # convert to scf oil/scf gas

  # oil FVF at surface/standard condition using Standing correlation (Equation 2.37 See Unit 2)

  F = Rs * ((sg / so)**0.5) + (1.25 * temp_sc) # Rs must be in scf/STB
  Bo = 0.972 + (0.000147*(F**1.1756))

  # oil density at surface/standard condition

  rhowater = 62.366 # 1 g/cm3 = 62.366 lbm/ft3
  rhooil_sc = so * rhowater

  # gas density at surface/standard condition (Eq 2.23, real-gas law)

  R = 10.732 # gas constant in (ft3*psi)/(lb-mol*R)
  rhogas_sc = (28.97 * sg * pressure_sc) / (z_sc * R * (temp_sc + 459)) # temp converted to Rankine

  # finally, oil density at reservoir condition

  rhooil = (rhooil_sc + (rhogas_sc * Rs_converted)) / Bo

  # oil density gradient

  rhooil_grad = rhooil / 144 # 144 is factor conversion from density lbm/ft3 to psi/ft
  rhooil_grad_converted = rhooil_grad * (6.89476 / 0.3048) # convert from psi/ft to kPa/m
  return(rhooil_grad, rhooil_grad_converted)

rhooil_grad, rhooil_grad_converted = rhooilgrad(so, sg, Rs, Bo)
print("Oil density gradient for case 1:", rhooil_grad, "psi/ft, or:", rhooil_grad_converted, "kPa/m")

Oil density gradient for case 1: 0.2844502289972827 psi/ft, or: 6.434435895279871 kPa/m


In [11]:
# Question 2. Determine oil density gradient for case 2

# known

so = 0.9
sg = 0.75
Rs = 940 # scf/STB
Bo = 1.63

rhooil_grad, rhooil_grad_converted = rhooilgrad(so, sg, Rs, Bo)
print("Oil density gradient for case 2:", rhooil_grad, "psi/ft, or:", rhooil_grad_converted, "kPa/m")

Oil density gradient for case 2: 0.31978837177053765 psi/ft, or: 7.233806017548005 kPa/m


In [12]:
# Question 3. Determine oil density gradient for case 2

# known

so = 0.8
sg = 0.65
Rs = 760 # scf/STB
Bo = 1.43

rhooil_grad, rhooil_grad_converted = rhooilgrad(so, sg, Rs, Bo)
print("Oil density gradient for case 3:", rhooil_grad, "psi/ft, or:", rhooil_grad_converted, "kPa/m")

Oil density gradient for case 3: 0.30401103004586716 psi/ft, or: 6.87691302335644 kPa/m


# Problem 3.2

In [22]:
# Question. Are the two wells in pressure communication?

# known

api = 38
Rs = 510 # in scf/STB
sg = 0.89
Bo = 1.3 # in RB/STB

well1_elev = 1202; well2_elev = 1098 # well elevation in ft MSL
well1_depth = 4608; well2_depth = 4572 # well depth from KB in ft
well1_pressure = 2108; well2_pressure = 2128 # well pressure in psia

# standard conditions

temp_sc = 60 # in fahrenheit
pressure_sc = 14.73 # 1 atm = 14.73 psia
z_sc = 1 # gas z factor at standard condition

# calculate oil density at standard condition

so = 141.5 / (131.5 + api)
rhowater = 62.366
rhooil_sc = so * rhowater
# print("Oil density at standard condition:", rhooil_sc, "lbm/ft3")

# calculate gas density at standard condition

R = 10.732 # gas constant in (ft3*psi)/(lb-mol*R)
rhogas_sc = (28.97 * sg * pressure_sc) / (z_sc * R * (temp_sc + 459)) # temp converted to Rankine
# print("Gas density at standard condition:", rhogas_sc, "lbm/ft3")

# calculate oil density at reservoir condition

Rs_converted = Rs * (1 / 5.6148) # convert to scf gas/scf oil
rhooil = (rhooil_sc + (rhogas_sc * Rs_converted)) / Bo
print("Oil density at reservoir condition:", rhooil, "lbm/ft3")

# oil density gradient

rhooil_grad = rhooil / 144 # 144 is factor conversion from density lbm/ft3 to psi/ft
# print("Oil density gradient:", rhooil_grad, "psi/ft")

# check if the two wells are communicated

well1_depthmsl = well1_elev - well1_depth
well2_depthmsl = well2_elev - well2_depth
delta_depth = np.abs(well2_depthmsl - well1_depthmsl)

if well1_depthmsl > well2_depthmsl:
  print("well 1 is shallower", delta_depth, "ft than well 2")
else:
  print("well 1 is deeper", delta_depth, "ft than well 2")

well1_pressure_extrapolated = well2_pressure + rhooil_grad * delta_depth # pressure extrapolation, Eq 3.1
print("From extrapolation, well 1 is supposed to be", well1_pressure_extrapolated, "psi")

delta_extrapolated_actual = np.abs(well1_pressure_extrapolated - well1_pressure)
percent_delta = delta_extrapolated_actual / well1_pressure * 100

print("Difference of extrapolated well 1 pressure to actual pressure:", delta_extrapolated_actual, "psi, or percent difference:", percent_delta, "%")

if percent_delta < 1:
  print("The two wells are most likely in the same reservoir because pressure difference < significance 1%")
else:
  print("The two wells are most likely NOT in the same reservoir because pressure difference > significance 1%")

Oil density at reservoir condition: 44.81311666129863 lbm/ft3
well 1 is shallower 68 ft than well 2
From extrapolation, well 1 is supposed to be 2149.161749534502 psi
Difference of extrapolated well 1 pressure to actual pressure: 41.16174953450218 psi, or percent difference: 1.9526446648245814 %
The two wells are most likely NOT in the same reservoir because pressure difference > significance 1%


# Problem 3.3

In [23]:
# Question 1. Calculate density gradient of stock-tank oil (standard condition) and in reservoir

# known

api = 38.1
Rs = 740 # in scf/STB
sg = 0.85
Bo = 1.451 # in RB/STB

# standard conditions

temp_sc = 60 # in fahrenheit
pressure_sc = 14.73 # 1 atm = 14.73 psia
z_sc = 1 # gas z factor at standard condition

# calculate oil density at standard condition

so = 141.5 / (131.5 + api)
rhowater = 62.366
rhooil_sc = so * rhowater
# print("Oil density at standard condition:", rhooil_sc, "lbm/ft3")

# calculate gas density at standard condition

R = 10.732 # gas constant in (ft3*psi)/(lb-mol*R)
rhogas_sc = (28.97 * sg * pressure_sc) / (z_sc * R * (temp_sc + 459)) # temp converted to Rankine
# print("Gas density at standard condition:", rhogas_sc, "lbm/ft3")

# calculate oil density at reservoir condition

Rs_converted = Rs * (1 / 5.6148) # convert to scf gas/scf oil
rhooil = (rhooil_sc + (rhogas_sc * Rs_converted)) / Bo
# print("Oil density at reservoir condition:", rhooil, "lbm/ft3")

# reservoir oil density gradient

rhooil_sc_grad = rhooil_sc / 144 # 144 is factor conversion from density lbm/ft3 to psi/ft
print("Density gradient of stock-tank oil:", rhooil_sc_grad, "psi/ft")

# reservoir oil density gradient

rhooil_grad = rhooil / 144 # 144 is factor conversion from density lbm/ft3 to psi/ft
print("Density gradient of oil in reservoir:", rhooil_grad, "psi/ft")

Density gradient of stock-tank oil: 0.36133995839884697 psi/ft
Density gradient of oil in reservoir: 0.2901043719107 psi/ft


In [24]:
# Question 2. Are the two wells in pressure communication?

well1_elev = 5200; well2_elev = 5098 # well elevation in ft MSL
well1_depth = 8838; well2_depth = 8719 # well depth from KB in ft
well1_pressure = 3255; well2_pressure = 3226 # well pressure in psia

well1_depthmsl = well1_elev - well1_depth
well2_depthmsl = well2_elev - well2_depth
delta_depth = np.abs(well2_depthmsl - well1_depthmsl)

if well1_depthmsl > well2_depthmsl:
  print("well 1 is shallower", delta_depth, "ft than well 2")
else:
  print("well 1 is deeper", delta_depth, "ft than well 2")

well1_pressure_extrapolated = well2_pressure + rhooil_grad * delta_depth # pressure extrapolation, Eq 3.1
print("From extrapolation, well 1 is supposed to be", well1_pressure_extrapolated, "psi")

delta_extrapolated_actual = np.abs(well1_pressure_extrapolated - well1_pressure)
percent_delta = delta_extrapolated_actual / well1_pressure * 100

print("Difference of extrapolated well 1 pressure to actual pressure:", delta_extrapolated_actual, "psi, or percent difference:", percent_delta, "%")

if percent_delta < 1:
  print("The two wells are most likely in the same reservoir because pressure difference < significance 1%")
else:
  print("The two wells are most likely NOT in the same reservoir because pressure difference > significance 1%")

well 1 is deeper 17 ft than well 2
From extrapolation, well 1 is supposed to be 3230.931774322482 psi
Difference of extrapolated well 1 pressure to actual pressure: 24.068225677518058 psi, or percent difference: 0.7394232158991723 %
The two wells are most likely in the same reservoir because pressure difference < significance 1%


# Problem 3.4 

In [4]:
# Question 1. determine z factor (non-sour gas)

# known

sg = 0.69 
pressure = 3416
temp = 195
x_h2s = 0
x_co2 = 0

import sys, os
sys.path.append('/content/reservoir-engineering/Unit 2 Review of Rock and Fluid Properties/functions')

from pseudoprops import pseudoprops

P_pr, T_pr = pseudoprops(temp, pressure, sg, x_h2s, x_co2)

from dranchuk_aboukassem import dranchuk

rho_pr, z = dranchuk(T_pr, P_pr)
print("Gas compressibility z factor:", z)

Gas compressibility z factor: 0.9023620730438874


In [9]:
# Question 2. calculate gas density gradient and pressure at 100 ft below datum

delta = 100
R = 10.732

rhogas = (28.97 * sg * pressure) / (z * R * (temp + 459)) # temp convert to Rankine

# gas density gradient

rhogas_grad = rhogas / 144
print("Gas density gradient:", rhogas_grad, "psi/ft")

# extrapolate using Eq 3.1

pressure_extrapolated_below = pressure + rhogas_grad * delta
print("Using Eq 3.1, extrapolated pressure at", delta, "ft below datum:", pressure_extrapolated_below, "psi")

Gas density gradient: 0.07487101318876115 psi/ft
Using Eq 3.1, extrapolated pressure at 100 ft below datum: 3423.4871013188763 psi


In [12]:
# Question 3. calculate surface shut-in pressure at 7500 ft above datum using Eq 3.1 and 3.7

# Eq 3.1

delta2 = 7500
pressure_extrapolated_above1 = pressure - rhogas_grad * delta
print("Using Eq 3.1, extrapolated pressure at", delta2, "ft above datum:", pressure_extrapolated_above1, "psi")

# Eq 3.7

pressure_extrapolated_above2 = pressure*(np.exp(-(0.01877 * sg * delta) / (z * (temp + 459)))) # temp in Rankine
print("Using Eq 3.7, extrapolated pressure at", delta2, "ft above datum:", pressure_extrapolated_above2, "psi")

Using Eq 3.1, extrapolated pressure at 7500 ft above datum: 3408.5128986811237 psi
Using Eq 3.7, extrapolated pressure at 7500 ft above datum: 3408.51147678938 psi
